In [5]:
from DSSATTools import (
    crop, # module that contains the classes for the different crops.
    WeatherStation, # class needed to define the Weather. 
    SoilProfile, SoilLayer, # classes that define the entire soil profile, and the soil layers within the soil profile
    filex, # module that contains the clases for the FileX sections.
    DSSAT # the simulation environment class.
)
from DSSATTools.filex import IrrigationEvent, FertilizerEvent, MowEvent # Individual events
import pandas as pd
from datetime import datetime
import tempfile
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import os
import shutil
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import Optional, Dict
import logging
import sys
import time
from typing import List

In [6]:
LATITUDE = 12.3811
LONGITUDE = 78.9366
COUNTRY = "India"


In [9]:
WEATHER_FILE = r'../randomized_env_workspace/L117519001.WTH'
SOIL_ID = "SG10857802"
SOIL_FILE = r'../randomized_env_workspace/soil10857802.SOL'
PLANTING_DATE = datetime(2014, 5, 2)
TODAY = 2
reward_factors={
    'GROWTH_FACTOR': 0.01,
    'WATER_COST_FACTOR': 2.0,
    'STRESS_PENALTY_FACTOR': 1.0,
    'N_STRESS_FACTOR': 0.5,
    'DRAINAGE_PENALTY_FACTOR': 0.5,
    'DRYNESS_PENALTY_FACTOR': 0.5,
    'DRYNESS_THRESHOLD': 0.4,
    'STEP_COST': 0.1
}


In [10]:
weather_station =  WeatherStation.from_files([WEATHER_FILE])

soil_profile = SoilProfile.from_file(SOIL_ID, SOIL_FILE)

cultivar = crop.Sorghum('IB0026')

field = filex.Field(id_field='REALTIME', wsta=weather_station, id_soil=soil_profile)

planting = filex.Planting(pdate=PLANTING_DATE, ppop=18, ppoe=18, plrs=45, pldp=5)

soil_depths = [int(layer['slb']) for layer in soil_profile.table]

# MODIFIED: Reduce initial soil water and use realistic irrigation schedule
initial_values_table = [(depth, 0.20, 1.5, 1.5) for depth in soil_depths]  # Reduced from 0.35 to 0.20
initial_conditions = filex.InitialConditions(
    pcr='SG', icdat=PLANTING_DATE, icres=1300, icren=0.5,
    table=pd.DataFrame(initial_values_table, columns=['icbl', 'sh2o', 'snh4', 'sno3'])
)

simulation_controls = filex.SimulationControls(
    general=filex.SCGeneral(sdate=PLANTING_DATE),
    options=filex.SCOptions(water='Y', nitro='N', symbi='N'),
    methods=filex.SCMethods(infil='S'),
    management=filex.SCManagement(irrig='R', ferti='N', resid='N', harvs='M'),
    outputs=filex.SCOutputs(grout='Y', waout='Y', niout='Y',ovvew='Y')
)

In [11]:
# irrigation_events = [
#     filex.IrrigationEvent(idate=PLANTING_DATE + timedelta(days=day), 
#                          irop='IR001', irval=15) 
#     for day in range(10, 90, 10)  # Every 10 days, 15mm each
# ]
irrigation_events = [
    filex.IrrigationEvent(idate=PLANTING_DATE , irop="IR001", irval=10),
    filex.IrrigationEvent(idate=PLANTING_DATE+timedelta(days=3) , irop="IR001", irval=10),
    filex.IrrigationEvent(idate=PLANTING_DATE+timedelta(days=5) , irop="IR001", irval=10),
                     ]

In [18]:
# --- 2. RUN SIMULATION ---
def run_simulation(irrigation_events):
    dssat_output = None
    irrigation = filex.Irrigation(
        efir=1, idep=30, iame='IR001',
        table=irrigation_events)
    try:
        workspace = os.path.join("dummy", "dssat_realistic_run")
        os.makedirs(workspace, exist_ok=True)
        
        shutil.copy(WEATHER_FILE, workspace)
        shutil.copy(SOIL_FILE, workspace)
        
        dssat = DSSAT(workspace)
        print(f"🚀 Running simulation in: {workspace}")
        
        results = dssat.run_treatment(
            field=field, cultivar=cultivar, planting=planting,
            initial_conditions=initial_conditions, irrigation=irrigation,
            simulation_controls=simulation_controls
        )
        print("✅ Simulation completed successfully.")
        dssat_output = dssat.output_tables
        
    except Exception as e:
        print(f"❌ An error occurred during the DSSAT run: {e}")
    return dssat_output

dssat_output = run_simulation(irrigation_events=irrigation_events)

dummy\dssat_realistic_run created.
🚀 Running simulation in: dummy\dssat_realistic_run

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1 -99 -99     0     0     2    10     4    17     0   -99      0   44
✅ Simulation completed successfully.


In [14]:

def get_state(dssat_output, current_day):
    
    # This line calculates the daily drainage for all days
    dssat_output['SoilWat']['DRNC_D'] = dssat_output['SoilWat']['DRNC'].diff().fillna(0)

    # create daily Precitpation because in output we can only see the cumulative 
    dssat_output['SoilWat']['DPREC'] = dssat_output['SoilWat']['PREC'].diff().fillna(
        dssat_output['SoilWat']['PREC'].iloc[0]
    )


    dssat_output['SoilWat']['Rainfall7DaySum'] = dssat_output['SoilWat']['DPREC'].rolling(window=7, min_periods=1).sum()
    dssat_output['SoilWat']['RainTodayFlag'] = dssat_output['SoilWat']['DPREC'].apply(lambda x: 1 if x > 0 else 0)
    # Initialize list to store days since last rain
    days_since_last_rain_list = []

    counter = None
    for rain_flag in dssat_output['SoilWat']['RainTodayFlag']:
        if rain_flag == 1:
            counter = 0
        else:
            counter = counter + 1 if counter is not None else None
        days_since_last_rain_list.append(counter)

    # Assign list to new column
    dssat_output['SoilWat']['DaysSinceLastRain'] = days_since_last_rain_list

    # Drop helper column
    dssat_output['SoilWat'].drop(columns='RainTodayFlag', inplace=True)

    current_weather =  dssat_output['Weather'][dssat_output['Weather']['DAS'] == current_day]
    current_plantgro =  dssat_output['PlantGro'][dssat_output['PlantGro']['DAS'] == current_day]
    current_soilorg =  dssat_output['SoilOrg'][dssat_output['SoilOrg']['DAS'] == current_day]
    current_soilwat =  dssat_output['SoilWat'][dssat_output['SoilWat']['DAS'] == current_day]
    # sw_cols = [col for col in dssat_output['SoilWat'].columns if re.match(r'^SW\d+D?$', col)]
    
    sw_cols = [col for col in dssat_output['SoilWat'].columns if col.startswith("SW") and col.endswith("D") and col[2:-1].isdigit()]
    
    
    def get_sw_by_depth(l,h):
        results = []
        for a,b in zip(sw_cols, soil_depths):
            if b > l and b <= h:
                results.append(current_soilwat[a])
        return (sum(results)/len(results)).iloc[0]

    # getting the last irrigation data
    days_since_last_irrigation = 0
    for irr in irrigation_events:
        if (irr.get('idate') - PLANTING_DATE.date()).days <= current_day:
            days_since_last_irrigation = current_day - (irr.get('idate') - PLANTING_DATE.date()).days 
        else:
            break;
    forecast = None
    try:
        forecast = dssat_output['SoilWat'][dssat_output['SoilWat']['DAS'] == current_day+1]['DPREC'].iloc[0],
        forecast = forecast[0]
    except:
        forecast = 0
    state = {
            'DAP': current_day,
            'days_since_last_rain': current_soilwat['DaysSinceLastRain'].iloc[0],
            'rainfall_7day': current_soilwat['Rainfall7DaySum'].iloc[0],
            'HarvestIndex' : current_plantgro['HIAD'].iloc[0],
            'phenological_stage': current_plantgro['GSTD'].iloc[0],
            'leaf_area_index': current_plantgro['LAID'].iloc[0],
            'total_biomass': current_plantgro['CWAD'].iloc[0],
            'soil_water_content_0_30cm': get_sw_by_depth(0, 30),
            'soil_water_content_30_60cm': get_sw_by_depth(30, 60),
            'soil_water_content_60_100cm': get_sw_by_depth(60, 100),
            'available_water_fraction': current_soilwat['SWXD'].iloc[0],
            'water_stress_factor': current_plantgro['WSGD'].iloc[0],
            'temperature_avg': current_weather['TAVD'].iloc[0],
            'solar_radiation': current_weather['SRAD'].iloc[0],
            'last_irrigation_amount': irrigation_events[-1].get('irval'),
            'days_since_last_irrigation': days_since_last_irrigation,
            'cumulative_irrigation': current_soilwat['IRRC'].iloc[0],
            'totalnumberIrrigation' : current_soilwat['IR#C'].iloc[0],
            'forecast' : forecast,
            'harverst_grain_weight' : dssat_output['PlantGro']['GWAD'].iloc[-1],
            'daily_drainage': current_soilwat['DRNC_D'].iloc[0]
        }
    return state    

In [19]:
dssat_output['PlantGro']

,@YEAR,DOY,DAS,DAP,L#SD,GSTD,LAID,LWAD,SWAD,GWAD,...,RL7D,RL8D,RL9D,RL10D,CDAD,LDAD,SDAD,SNW0C,SNW1C,DTTD
2114-05-02,2114,122,1,0,0.0,0,0.0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,20.25


In [15]:
day42 = get_state(dssat_output=dssat_output, current_day = 42) 
print(day42)
print(len(day42))

IndexError: single positional indexer is out-of-bounds

In [ ]:
for i in range(1,20):
    print(get_state(dssat_output=dssat_output, current_day = i))
    

{'DAP': 1, 'days_since_last_rain': np.float64(0.0), 'rainfall_7day': np.float64(2.0), 'HarvestIndex': np.float64(0.0), 'phenological_stage': np.int64(0), 'leaf_area_index': np.float64(0.0), 'total_biomass': np.int64(0), 'soil_water_content_0_30cm': np.float64(0.23666666666666666), 'soil_water_content_30_60cm': np.float64(0.202), 'soil_water_content_60_100cm': np.float64(0.202), 'available_water_fraction': np.int64(20), 'water_stress_factor': np.float64(0.0), 'temperature_avg': np.float64(31.7), 'solar_radiation': np.float64(23.3), 'last_irrigation_amount': 10.0, 'days_since_last_irrigation': 1, 'cumulative_irrigation': np.int64(10), 'totalnumberIrrigation': np.int64(1), 'forecast': np.float64(1.0), 'harverst_grain_weight': np.int64(1962), 'daily_drainage': np.float64(0.0)}
{'DAP': 2, 'days_since_last_rain': np.float64(0.0), 'rainfall_7day': np.float64(3.0), 'HarvestIndex': np.float64(0.0), 'phenological_stage': np.int64(0), 'leaf_area_index': np.float64(0.0), 'total_biomass': np.int64(

In [ ]:
def calculate_reward(previous_state, current_state, action, reward_factors) -> float:
    """Calculates the reward based on the designed multi-component function."""
    if not current_state:
        return -1000
    
    # 1. Growth Reward
    prev_biomass = previous_state['total_biomass'] if previous_state else current_state['total_biomass']
    daily_growth_reward = (current_state['total_biomass'] - prev_biomass) * reward_factors['GROWTH_FACTOR']
    final_harvest_bonus = current_state['harverst_grain_weight']/1000 if current_state['phenological_stage'] == 7 else 0

    # 2. Water Cost Penalty
    water_cost_penalty = action * reward_factors['WATER_COST_FACTOR']

    # 3. Stress & Waste Penalties
    water_stress_penalty = current_state['water_stress_factor'] * reward_factors['STRESS_PENALTY_FACTOR']
    drainage_penalty = current_state['daily_drainage'] * reward_factors['DRAINAGE_PENALTY_FACTOR']

    # 4. Root Zone Dryness Penalty
    dryness_penalty = 0
    slll_top = soil_profile.table[0]['slll']
    sdul_top = soil_profile.table[0]['sdul']
    if sdul_top > slll_top and slll_top != -99:
        pawf = (current_state['soil_water_content_0_30cm'] - slll_top) / (sdul_top - slll_top)
        if pawf < reward_factors['DRYNESS_THRESHOLD']:
            dryness_penalty = (reward_factors['DRYNESS_THRESHOLD'] - pawf) * reward_factors['DRYNESS_PENALTY_FACTOR']

    # 5. Efficiency Penalty

    total_reward = (
        daily_growth_reward +
        final_harvest_bonus -
        (water_cost_penalty + water_stress_penalty  + drainage_penalty + dryness_penalty)
    )
    return float(total_reward)

i = 2
day1 = get_state(dssat_output=dssat_output, current_day=1)
total = pd.DataFrame([day1], index=[1])

while True:
    cur = get_state(dssat_output=dssat_output, current_day=i+1)
    prev = get_state(dssat_output=dssat_output, current_day=i)
    
    re = calculate_reward(
        previous_state=prev,
        current_state=cur,
        action=10,
        reward_factors=reward_factors
    )
    print(re)
    total = pd.concat([total, pd.DataFrame([cur])], ignore_index=True)

    if cur.get('phenological_stage') == 7:
        break
    i += 1


-20.054785478547856
-20.0
-20.0
-19.97
-20.0
-20.0
-19.99
-20.0
-19.98
-19.99
-19.98
-19.98
-19.97
-19.96
-19.95
-19.93
-19.92
-19.9
-19.89
-19.87
-19.85
-19.41
-19.23
-18.98
-18.6
-18.29
-17.87
-17.951999999999998
-19.533732673267327
-19.827
-17.85
-17.68
-17.53
-17.37
-17.16
-17.48
-18.89
-18.1
-16.96
-17.352135313531353
-19.33073597359736
-19.888587458745874
-20.125089108910892
-20.435940594059403
-20.513141914191422
-20.51404290429043
-20.589293729372937
-20.666742574257427
-20.485999999999997
-17.484534653465346
-19.753587458745876
-20.2479900990099
-20.57079207920792
-20.648643564356437
-20.73749504950495
-20.57
-17.375
-16.93
-16.88
-17.771782178217823
-19.31713861386139
-20.330891089108913
-20.322
-18.542438943894393
-20.30308910891089
-20.297541254125417
-20.467036303630362
-19.30428382838284
-18.826039603960396
-20.326240924092406
-20.41708580858086
-19.04974587458746
-20.81894719471947
-20.906749174917493
-20.98645214521452
-21.02390429042904
-21.09710561056106
-21.061306930

In [ ]:
total

,DAP,days_since_last_rain,rainfall_7day,HarvestIndex,phenological_stage,leaf_area_index,total_biomass,soil_water_content_0_30cm,soil_water_content_30_60cm,soil_water_content_60_100cm,...,water_stress_factor,temperature_avg,solar_radiation,last_irrigation_amount,days_since_last_irrigation,cumulative_irrigation,totalnumberIrrigation,forecast,harverst_grain_weight,daily_drainage
0,1,0.0,2.0,0.000,0,0.00,0,0.236667,0.202,0.202,...,0.000,31.7,23.3,10.0,1,10,1,1.0,1962,0.0
1,3,0.0,6.0,0.000,0,0.00,0,0.209333,0.202,0.202,...,0.000,30.3,22.3,10.0,0,10,1,3.0,1962,0.0
2,4,0.0,9.0,0.000,0,0.00,0,0.254333,0.202,0.202,...,0.000,29.4,20.5,10.0,1,20,2,11.0,1962,0.0
3,5,0.0,20.0,0.000,0,0.00,0,0.279667,0.202,0.202,...,0.000,26.9,14.3,10.0,0,20,2,14.0,1962,0.0
4,6,0.0,34.0,0.000,0,0.00,3,0.320667,0.231,0.202,...,0.000,26.4,14.3,10.0,1,30,3,8.0,1962,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,91,1.0,8.0,0.268,5,1.33,7289,0.166333,0.202,0.202,...,0.989,28.1,15.8,10.0,86,30,3,1.0,1962,0.0
90,92,0.0,9.0,0.268,5,1.26,7293,0.167000,0.202,0.202,...,0.974,28.7,16.2,10.0,87,30,3,0.0,1962,0.0
91,93,1.0,3.0,0.269,5,1.20,7295,0.163667,0.202,0.202,...,0.981,29.1,9.4,10.0,88,30,3,1.0,1962,0.0
92,94,0.0,3.0,0.269,6,1.20,7295,0.169000,0.202,0.202,...,0.981,28.8,15.8,10.0,89,30,3,1.0,1962,0.0
